In [355]:
import bson
from datetime import datetime
import json
import random
from faker import Faker
from tqdm import tqdm

In [356]:
fake = Faker('es_ES')

Generar habitaciones

In [357]:
room_ids = []

rooms_dict = {}

room_type_prices = {'suite presidencial': 350000, 'suite': 250000, 'familiar': 450000, 'doble': 200000, 'sencilla': 100000}

room_type_capacity = {'suite presidencial': 2, 'suite': 3, 'familiar': 6, 'doble': 4, 'sencilla': 2}

room_types = ['suite presidencial', 'suite', 'familiar', 'doble', 'sencilla']

room_type_amenities = {
    'suite presidencial': ['aire acondicionado', 'calefacción', 'cocina', 'comedor', 'escritorio', 'caja de seguridad', 'jacuzzi', 'cafetera', 'balcon', 'televisor', 'minibar'], 
    'suite': ['cafetera', 'caja de seguridad', 'balcon', 'jacuzzi', 'aire acondicionado', 'calefaccion', 'cocina', 'televisor', 'minibar'],
    'familiar': ['comedor', 'cafetera', 'caja de seguridad', 'aire acondicionado', 'calefaccion', 'cocina', 'televisor', 'minibar'],
    'doble': ['cafetera', 'caja de seguridad', 'aire acondicionado', 'calefaccion' 'televisor', 'minibar'],
    'sencilla': ['cafetera', 'caja de seguridad', 'aire acondicionado', 'calefaccion', 'televisor', 'minibar']
    }

def random_letter():
    return random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

def random_3_digit_number():
    return random.randint(100, 999)

def random_room_id():
    return f'{random_letter()}{random_3_digit_number()}'

def create_random_room(room_ids, room_type_prices, room_type_capacity, room_types, room_type_amenities, rooms_dict):
    
    room_id = random_room_id()
    while room_id in room_ids:
        room_id = random_room_id()
    room_ids.append(room_id)
        
    room_type = random.choice(room_types)
    
    room = {
        '_id': room_id,
        'tipo': {
                'tipo': room_type,
                'costo_por_noche': room_type_prices[room_type],
                'capacidad': room_type_capacity[room_type],
                'dotaciones': room_type_amenities[room_type]
            },
        'reservas': []
    }
    
    rooms_dict[room_id] = room
    
    return room

Generar usuarios

In [358]:
user_ids = []

users_dict = {}

user_types = ['admin', 'gerente', 'cliente', 'recepcionista', 'empleado']

def random_name():
    return fake.name()

def random_email():
    return fake.email()

def random_password():
    return fake.lexify(text='????????')

def random_doc_num():
    return fake.random_number(8)

def random_doc_type():
    return random.choice(['CC', 'PA', 'CE'])

def create_random_user(user_ids, user_types, users_dict):
    
    while True:
        tipo_doc = random_doc_type()
        num_doc = random_doc_num()
        if (tipo_doc, num_doc) not in user_ids:
            user_ids.append((tipo_doc, num_doc))
            break
        
    user_type = random.choice(user_types)
    
    user = {
        'tipo_doc': tipo_doc,
        'num_doc': num_doc,
        'tipo': user_type,
        'nombre': random_name(),
        'email': random_email(),
        'password': random_password(),
        'reservas_habitaciones': [],
        'reservas_servicios': []
    }
    
    users_dict[f'{tipo_doc}{num_doc}'] = user
    
    return user

Generar servicios

In [359]:
service_oids = []

services_dict = {}

service_names = ['Jazz bar', 'Rock bar', 'Salsa bar', 'Crossover bar', 'Restaurante iteliano', 'Restaurante japones', 'Restaurante de carnes', 'Restaurante vegetariano', 'Restaurante de comida rapida', 'Restaurante de comida tipica', 'Restaurante mediterraneo', 'Restaurante frances', 'Restaurante mexicano', 'Restaurante desayunos', 'Restaurante Buffet', 'Drogueria', 'Gimnasio cardio', 'Gimnasio Fuerza', 'Internet basico', 'Internet medio', 'Internet preferencial', 'Lavanderia', 'Piscina', 'Prestamo utensilios deportivos', 'Prestamo de toallas', 'Prestamo juegos de mesa', 'Salon de conferencias', 'Salon de reuniones', 'Spa deportista', 'Spa premium', 'Spa relax', 'Spa basico', 'Supermercado', 'Tienda de deportes', 'Tienda de joyeria', 'Tienda de souvenirs', 'Tienda de ropa']

def generate_oid():
    return bson.ObjectId()

def create_services(service_oids, service_names, services_dict):
    
    for service_name in service_names:
        oid = str(generate_oid())
        service_oids.append(oid)
        service = {
            '_id': {
                "$oid": oid
            },
            'nombre': service_name,
        }
        
        if 'bar' in service_name:
            service['capacidad'] = random.randint(10, 50)
            service['horario'] = 'nocturno'
        elif 'Restaurante' in service_name:
            service['capacidad'] = random.randint(50, 200)
            service['horario'] = 'almuerzo y cena'
        elif 'Drogueria' in service_name:
            service['horario'] = '24 horas'
        elif 'Gimnasio' in service_name:
            service['horario'] = 'diurno'
        elif 'Lavanderia' in service_name:
            service['horario'] = 'diurno'
        elif 'Piscina' in service_name:
            service['horario'] = 'diurno'
        elif 'Prestamo' in service_name:
            service['horario'] = 'diurno'
        elif 'Salon' in service_name:
            service['horario'] = 'diurno'
            service['capacidad'] = random.randint(10, 200)
        elif 'Spa' in service_name:
            service['horario'] = 'diurno'
        elif 'Supermercado' in service_name:
            service['horario'] = '24 horas'
        elif 'Tienda' in service_name:
            service['horario'] = 'diurno'
            
        service['reservas'] = []
        
        services_dict[oid] = service

Generar reservas de habitaciones

In [360]:
room_reservations_ids = []
room_reservations_dict = {}

planes_consumo_dict = {
    'larga estadia': {
        'tasa': 1.01,
        'min_noches': 7,
        'max_noches': 20
        },
    'todo incluido': {
        'tasa': 1.05,
        'min_noches': 1,
        'max_noches': 10
        },
    'tiempo compartido': {
        'tasa': 1.03,
        'min_noches': 5,
        'max_noches': 7
        },
    'promocion particular': {
        'tasa': 0.98,
        'min_noches': 1,
        'max_noches': 10
        },
    'sin plan': {
        'tasa': 1.0,
        'min_noches': 1,
        'max_noches': 365
        }
}

planes_consumo = list(planes_consumo_dict.keys())

def random_date(initial_date='2017-01-01', final_date='2023-12-01'):
    
    date_between = fake.date_between(start_date=datetime.strptime(initial_date, '%Y-%m-%d'), end_date=datetime.strptime(final_date, '%Y-%m-%d'))
    
    return date_between.isoformat()

def days_between_dates(start_date_str, end_date_str):
    date_format = '%Y-%m-%d'  # Specify the format of the date strings

    # Convert the strings to datetime objects
    start_date = datetime.strptime(start_date_str, date_format)
    end_date = datetime.strptime(end_date_str, date_format)

    # Calculate the difference in days
    difference = end_date - start_date

    # Return the number of days (using .days attribute)
    return difference.days


def create_room_reservation(room_ids, user_ids, room_reservations_ids, room_reservations_dict, users_dict, rooms_dict, room_type_prices, room_type_capacity, planes_consumo, planes_consumo_dict):
    
    oid = str(generate_oid())
    room_reservations_ids.append(oid)
    
    room_id = random.choice(room_ids)
    user_id = random.choice(user_ids)
    
    room_type = rooms_dict[room_id]['tipo']['tipo']
    
    fecha_inicio = random_date()
    fecha_fin = random_date(initial_date=fecha_inicio)
    plan_consumo = random.choice(planes_consumo)
    room_reservation = {
        '_id': {
            '$oid': oid
            },
        'id_habitacion': room_id,
        'usuario': {
            'tipo_doc': user_id[0],
            'num_doc': user_id[1]
        },
        'fecha_inicio': {
            '$date': fecha_inicio
        },
        'fecha_fin': {
            '$date': fecha_fin
        },
        'num_personas': random.randint(1, room_type_capacity[room_type]),
        'plan_consumo': {
            'nombre': plan_consumo,
            'tasa': planes_consumo_dict[plan_consumo]['tasa'],
            'min_noches': planes_consumo_dict[plan_consumo]['min_noches'],
            'max_noches': planes_consumo_dict[plan_consumo]['max_noches']
            },
        'costo_reserva': room_type_prices[room_type] * planes_consumo_dict[plan_consumo]['tasa'] * days_between_dates(fecha_inicio, fecha_fin),
        'costo_consumos': random.randint(0, 1000000),
        'cuenta_abierta': random.choice([True, False]),
        'fecha_check_in': {
            '$date': fecha_inicio
        },
        'fecha_check_out': {
            '$date': fecha_fin
        },
    }
    
    users_dict[f'{user_id[0]}{user_id[1]}']['reservas_habitaciones'].append({'$oid': oid})
    rooms_dict[room_id]['reservas'].append({'$oid': oid})

    
    room_reservations_dict[oid] = room_reservation
    
    return room_reservation

Generar reservas de servicios

In [361]:
service_reservations = []

def create_service_reservation(services_dict, users_dict, room_reservations_dict, service_oids, user_ids, room_reservations_ids):
    
    oid = str(generate_oid())
    
    service_oid = random.choice(service_oids)
    room_reservation_oid = random.choice(room_reservations_ids)
    user_id = random.choice(user_ids)
    
    fecha = random_date(initial_date=room_reservations_dict[room_reservation_oid]['fecha_inicio']['$date'], final_date=room_reservations_dict[room_reservation_oid]['fecha_fin']['$date'])
    
    service_reservation = {
        '_id': {
            '$oid': oid
            },
        'id_servicio': {
            '$oid': service_oid,
        },
        'id_reserva_habitacion': {
            '$oid': room_reservation_oid
            },
        'usuario': {
            'tipo_doc': user_id[0],
            'num_doc': user_id[1]
        },
        'fecha': {
            '$date': fecha
        },
        'costo': random.randint(0, 600000)
    }
    
    users_dict[f'{user_id[0]}{user_id[1]}']['reservas_servicios'].append({'$oid': oid})
    services_dict[service_oid]['reservas'].append({'$oid': oid})
    service_reservations.append(service_reservation)
    
    return service_reservation

Crear los archivos _.json_ con los datos generados

In [362]:
for _ in tqdm(range(500)):
    create_random_room(room_ids, room_type_prices, room_type_capacity, room_types, room_type_amenities, rooms_dict)
    
for _ in tqdm(range(30000)):
    create_random_user(user_ids, user_types, users_dict)
    
create_services(service_oids, service_names, services_dict)

for _ in tqdm(range(60000)):
    create_room_reservation(room_ids, user_ids, room_reservations_ids, room_reservations_dict, users_dict, rooms_dict, room_type_prices, room_type_capacity, planes_consumo, planes_consumo_dict)

for _ in tqdm(range(100000)):
    create_service_reservation(services_dict, users_dict, room_reservations_dict, service_oids, user_ids, room_reservations_ids)


100%|██████████| 100000/100000 [00:05<00:00, 19981.70it/s]


In [363]:
with open('habitaciones.json', 'w') as f:
    json.dump([v for v in rooms_dict.values()], f)
    
with open('usuarios.json', 'w') as f:
    json.dump([v for v in users_dict.values()], f)
    
with open('servicios.json', 'w') as f:
    json.dump([v for v in services_dict.values()], f)
    
with open('reservas_habitaciones.json', 'w') as f:
    json.dump([v for v in room_reservations_dict.values()], f)
    
with open('reservas_servicios.json', 'w') as f:
    json.dump([sr for sr in service_reservations], f)